In [241]:
#Import des modules utilisée dans le code
import os
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerTuple
from scipy import fftpack as ft
from PIL import Image as im

In [242]:
#Installation du parent directory (à initialiser correctement)
os.chdir("/Users/Alexandre/Desktop/Memoire_L3 ")

In [243]:
#méta-classe image en couleur
class Image_couleur:
    def __init__(self,chemin):
        self.img=im.open(chemin)
        self.pix_val = list(self.img.getdata())
        self.luminance=[]
        self.chrominance1=[]
        self.chrominance2=[]
    def divisionen3(self):
        if type(self.pix_val[0])==tuple:
            for i in self.pix_val:
                self.luminance.append(np.mean(i))
                self.chrominance1.append(i[0]/2+i[1]/2)
                self.chrominance2.append(i[0]/2+i[2]/2)
            self.chrominance1=np.array(self.chrominance1)
            self.chrominance2=np.array(self.chrominance2)
            self.chrominance1=self.chrominance1.reshape(self.img.size)
            self.chrominance2=self.chrominance2.reshape(self.img.size)
        else:
            self.luminance=self.pix_val
        self.luminance=np.array(self.luminance)
        self.luminance=self.luminance.reshape(self.img.size[1],self.img.size[0]) 
        return self

In [244]:
#méta-classe image 
class Image:
    def __init__(self, nom, matrice):
        self.nom = nom
        self.contenu = np.array(matrice) #matrice de l'image
        self.lenx = len(matrice[0]) #longueur
        self.leny = len(matrice) #hauteur
        self.blocs = [] #liste des blocs 8x8 de l'image
        self.rle=[] #concaténation des blocs
        self.huffman="" #Passage à la moulinette Huffman 
        self.dic={}
        self.namffuh=""
        self.erl=[] #Inversion du remplacement linéaire
        self.Resultat=[]
        self.coeff=[]
        self.cpt=[]
        self.erreur=0 #Metrique de qualité
        self.var=0

    def division(self):
        y=0
        cpt=1
        while y+8<=self.leny:
            x=0
            while x+8<=self.lenx:
                self.blocs.append(Bloc8x8(str(cpt),self.contenu[y:y+8,x:x+8]))
                cpt+=1
                x+=8
            if x!=self.lenx:
                L=[list(self.contenu[z,x:self.lenx])+[self.contenu[z,self.lenx-1] for i in range(8-(self.lenx-x))] for z in range(y,y+8)]
                self.blocs.append(Bloc8x8(str(cpt),np.array(L)))
                cpt+=1
            y+=8
        if y!=self.leny:
            x=0
            while x+8<=self.lenx:
                L=[list(self.contenu[z,x:x+8]) for z in range(y,self.leny)]+[list(self.contenu[self.leny-1,x:x+8]) for z in range(8-(self.leny-y))]
                self.blocs.append(Bloc8x8(str(cpt),np.array(L)))
                cpt+=1
                x+=8
            if x!=self.lenx:
                L=[]
                for z in range (y,self.leny):
                    L.append(list(self.contenu[z,x:self.lenx])+[self.contenu[z,self.lenx-1] for i in range(8-(self.lenx-x))])
                L=L+[L[-1] for z in range(8-(self.leny-y))]
                self.blocs.append(Bloc8x8(str(cpt),np.array(L)))         

    def compressI(self,Q):
        for b in self.blocs:
            b.compress(Q)
            self.rle.extend(b.rle)
        self.huffman,self.dic=huffman(self.rle)
        return self
        
    def recollage(self):
        M=0
        nx=self.lenx//8
        if self.lenx%8:
            nx+=1
        ny=self.leny//8
        if self.leny%8:
            ny+=1
        for y in range(ny):
            L=0
            for x in range(nx):
                if type(L)==int:
                    L=self.blocs[x+nx*y].comp
                else:
                    L=np.concatenate((L,self.blocs[x+nx*y].comp),axis=1)
            if type(M)==int:
                M=L
            else:
                M=np.concatenate((M,L),axis=0)
        self.Resultat=M[:self.leny,:self.lenx]
        return self
        
    def metric(self,k,i,B): 
        self.erreur=np.mean(abs((self.Resultat.astype(float)-self.contenu)))
        self.var=np.var((self.Resultat.astype(float)-self.contenu))
        if B:
            Table[k][i].append((round(len(self.huffman)/(self.lenx*self.leny * 8),3),round(self.erreur,3),round(self.var,3)))
        else:
            print("Avant : {} bits / Après : {} bits,\n Taux de compression: {}\nErreur Moyenne: {} \n Variance de l'erreur: {}".format(self.lenx*self.leny * 8, len(self.huffman),round(len(self.huffman)/(self.lenx*self.leny * 8),2),round(self.erreur,3),round(self.var,3)))


    def coeffcount(self):
        for j in range(max(self.coeff)):
            self.cpt.append(self.coeff.count(j))
        return self.cpt



    def decompressI(self,Q,showimage=False):
        self.namffuh=huffman_inverse(self.huffman,self.dic)
        for i in self.namffuh:
            if type(i)==list:
                self.erl.extend([0]*i[1])
            else :
                self.erl.append(i)
        
        self.coeff=self.erl
        self.erl=np.array(self.erl)
        self.erl=self.erl.reshape((len(self.erl)//64,64))
        i=0
        for b in self.blocs:
            b.zigzaged=self.erl[i]
            b.inv_zigzag()
            i+=1
        for b in self.blocs:
            b.decompress(Q)
            self.erreur=self.erreur+b.erreur
        self.erreur=self.erreur/i
        self.recollage()
        if showimage:
            self.show_full_image()

    def show_full_image(self): #Représente l'image 
            new_im = im.fromarray(abs((self.Resultat.astype(float)-self.contenu))*2) #Erreur
            new_im.show()
            if new_im.mode != 'RGB':
                new_im = new_im.convert('RGB')
            new_im.save("Résultat/"+self.nom+"_erreur_residuel.jpg")
            new_im = im.fromarray(self.Resultat.astype(float)) #Image comprimer
            new_im.show()
            if new_im.mode != 'RGB':
                new_im = new_im.convert('RGB')
            new_im.save("Résultat/"+self.nom+"_img_residuel.jpg")
            new_im = im.fromarray(self.contenu.astype(float)) # Image original
            new_im.show()
            return self


In [245]:
class Bloc8x8: #Classe sur un bloc  8 par 8
    def __init__(self, nom, matrice):  
        self.nom = nom 
        self.contenu = matrice #Bloc 8x8 avec lequel on travaille type np.array
        self.valide=True
        self.comp = [] # image comprimé
        self.DCT = np.array([])
        self.line = [] 
        self.rle=[] 
        self.zigzaged=[]
        self.dezigzaged=[]
        self.erreur=0    
        
    def test(self):
        print("Nom: ", self.nom, "type: ", type(self.nom))
        print("Contenu: ", self.contenu, "type: ", type(self.contenu))
        print("Valide: ", self.valide) 
        print("Compilation: ", self.comp, "type: ", type(self.comp))
        print("DCT: ", self.DCT, "type: ", type(self.DCT))
        print("Line: ", self.line, "type: ", type(self.line))
        print("RLE: ", self.rle, "type: ", type(self.rle))
    
    
    def check_matrice(self): #Vérifie la validité des données
        if self.contenu.shape != (8,8) :
            self.valide = False
            print("error")
        for i in range(0,8):
             for j in range(0,8):
                if self.contenu[i,j]> 255 or self.contenu[i,j]<0:
                    self.valide = False
                    print("error")
                    
    def show(self, trans): #Représente l'image, trans permet de choisir si on veut l'image comprimée ou pas
        self.check_matrice()
        if self.valide:
            if trans==True:
                plt.figure(figsize=(8, 8))
                #Afficher deux images côte à côte, format:(row,column,1er/2e image)
                plt.subplot(1,2,1) 
                plt.imshow(self.contenu, cmap="gray", vmin=0, vmax=255)
                plt.colorbar(label="Intensity (0-255)", fraction = 0.046)
                plt.title("Image originale " +self.nom)
                plt.axis("off")
                
                plt.subplot(1,2,2)
                plt.imshow(self.comp, cmap="gray", vmin=0, vmax=255)
                plt.colorbar(label="Intensity (0-255)", fraction = 0.046)
                plt.title("Image compressée " +self.nom)
                plt.axis("off")
            else:
                #plt.figure(figsize=(D, D))
                #plt.imshow(self.contenu, cmap="gray", vmin=0, vmax=255)
                #plt.colorbar(label="Intensity (0-255)", fraction = 0.046)
                #plt.title("Visualisation image " +self.nom)
                #plt.axis("off")
                pass           
            plt.tight_layout()
            plt.show()
            return self
        else:
            print("Matrice sous un mauvais format")
            
            
    def calcul(self,Q):
        self.DCT = ft.dct(ft.dct(self.contenu.T, norm='ortho').T,norm='ortho')
        self.DCT = np.round(self.DCT/Q).astype(int)
        return self.DCT
  
    def zigzag(self):
        # M np.array de de dimension d*d
        sgn=1 #sens de parcours de la diagonale
        back=False #mot-clef début de la seconde moitié de la matrice
        for sum in range(2*8 -1):
            #itération sur la somme de i abscisse et j ordonnée de chaque coefficient de M ie les diagonales de M
            if sum==8:
                back=True
            #détermination de la liste des abscisses des coefficients de la diagonale à parcourir
            if sgn==-1:
                if not back:
                    R=range(sum+1)
                else:
                    R=range(sum-8+1,8)
            else:
                if not back:
                    R=range(sum,-1,-1)
                else:
                    R=range(8-1,sum-8,-1)
            for i in R:
                self.line.append(self.DCT[i,sum-i]) #i+j=sum donc j=sum-i
            sgn=-sgn
        return self.line
    
    
    def RLE(self):
        k=-1 #marqueur initial k
        cpt=0
        for i in self.line: 
            if i==0:
                # cas initial (cas premier 0) pour i = 0: on crée la liste des 0
                if k!=0:
                    self.rle.append([0,1])
                    cpt+=1
                # 0 suivants
                else:
                    self.rle[cpt-1]=[0,self.rle[cpt-1][1]+1]
            # cas différent de 0
            else:
                self.rle.append(i)
                cpt+=1
            k=i
        return self
    
    def inv_zigzag(self):
        sgn=-1
        L=[[] for i in range(8)]
        back=False
        cpt=0
        for i in range(1,2*8):
            if i==9:
                back=True
            if not back:
                D={j:self.zigzaged[cpt+j] for j in range(i)}
                n=len(D)
                if sgn==-1:
                    for j in range(n):
                        L[j].append(D[n-j-1])
                else:
                    for j in range(i):
                        L[j].append(D[j])
            else:
                D={j:self.zigzaged[cpt+j] for j in range(8-i%8)}
                n=len(D)
                if sgn==1:
                    for j in range(n):
                        L[7-j].append(D[n-j-1])
                else:
                    for j in range(n):
                        L[7-j].append(D[j])
            cpt+=n
            sgn=-sgn
        self.dezigzaged=np.array(L)
        return self
    
    
    def inverse_calcul(self,Q):
        self.dequantifier = self.DCT*Q
        self.comp = ft.idct(ft.idct(self.dequantifier.T, norm='ortho').T,norm='ortho')
        return self.comp
  
    def score(self):
        self.erreur=(np.sum(abs(self.comp-self.contenu))/64)
        return self
    
    def compress(self,Q): #Fait les étapes une par une jusqu'à la compression
        #self.show(False)
        self.calcul(Q)
        self.zigzag()
        self.RLE()
        return self

    def decompress(self,Q): #Fait les étapes de décompression
        self.inverse_calcul(Q)
        self.score()
        #self.show(True)
        return self

In [246]:
#Huffman adapté de https://gist.github.com/GautierLePire/ee104b066f184f9c62bc89e87e494c06 par gaultier LePire
def compterOccurences(liste):
    """
    Renvoie une liste qui associe à chaque caractère son nombre d'apparitions.
    Chaque lettre est donc dottée d'un poids (son nombre d'occurences), et
    plus son poids est élevé, plus elle sera légère en mémoire.
    (L'objectif étant ici la compression, donc d'échanger de la puissance
    de calcul contre de l'espace de stockage.)
    """
    lettres={}
    for i in liste:
        j=i
        if type(i)==list:
            j="$"+str(i[1])
        if j in lettres :
            lettres[j]+=1
        else:
            lettres[j]=1
    alphabet=[]
    for i in lettres:
        alphabet.append([lettres[i],i])
    return alphabet

def creerArbre(alphabet):
    """
    Crée un arbre binaire à partir des lettres et de leur poids.
    On choisit de représenter un arbre de la façon suivante :
      * Une feuille est un 2-uplet : le nombre d'occurences et la lettre
        On notera que compterOccurences renvoie en fait une liste de feuilles.
      * Un noeud est un 3-uplet : la somme des occurences de toutes
        les feuilles descendantes, le fils gauche et le fils droit.
    Ensuite, on construit l'arbre en piochant les deux noeuds de poids
    le plus faible, on en fait un nouveau noeud que l'on remet dans le tas.
    On s'arrête dès qu'il reste un unique noeud (qui est l'arbre voulu)
    """
    # On commence par enlever les lettres qui ne sont pas présentes
    noeuds = alphabet
    # Puis on récupère les deux noeuds (ou feuilles) de poids le plus faible,
    # et on en fait un noeud, de poids la somme des deux petits poids
    # On boucle tant qu'il y a reste au moins deux noeuds
    l = len(noeuds)
    while l >= 2:
        # Indice et noeud des minima des poids
        # (on initialise avec les deux premières valeurs)
        petitMin = (0, noeuds[0])
        grandMin = (1, noeuds[1])
        for i in range(2, l):
            if noeuds[i][0] <= petitMin[1][0]:  # poids < petitMin < grandMin
                grandMin = petitMin
                petitMin = (i, noeuds[i])
            elif noeuds[i][0] <= grandMin[1][0]:  # petitMin < poids < grandMin
                grandMin = (i, noeuds[i])
        nouveauNoeud = (
            petitMin[1][0] + grandMin[1][0],
            noeuds[petitMin[0]],
            noeuds[grandMin[0]]
        )
        # On enlève les deux noeuds (ou feuilles) précedentes
        # et on ajoute le nouveau noeud
        noeuds[petitMin[0]] = nouveauNoeud
        noeuds.pop(grandMin[0])
        # On a au final un noeud de moins (-2 +1)
        l -= 1
    # À cet instant il ne reste plus qu'un noeud, qui est la racine de
    # l'arbre de Huffman
    return noeuds[0]

"""
On remarquera que moins une lettre est présente, plus elle est profonde dans l'arbre.
L'idée maintenant va être d'attribuer à chaque lettre un code en binaire de la façon
suivante :
  * la longueur du code est égale à la profondeur dans l'arbre
  * pour chaque lettre (donc chaque feuille), on part de la racine et on ajoute un 0
    si on prend la branche de gauche, un 1 si on passe par celle de droite.
Ainsi, plus une lettre est fréquente, plus son code binaire est court.
"""

def creerDico(arbre):
    """
    Renvoie un dictionnaire {lettre: code binaire}.
    On va explorer l'arbre à l'aide d'une file : si on rencontre une feuille,
    on la traite, si on rencontre un noeud, on ajoute les deux branches à la file.
    Le premier composant d'un élément de la file est le code binaire jusqu'à cet élément,
    le second est un noeud ou une feuille.
    """
    fileExploration = [("", arbre)]
    dico = {}
    l = 1
    # On boucle tant que la file n'est pas vide
    while l >= 1:
        code, truc = fileExploration.pop(0)  # On défile le premier élément
        l -= 1
        if len(truc) == 2:  # C'est une feuille
            dico[truc[1]] = code  # On ajoute la lettre et son code au dico
        elif len(truc) == 3:  # C'est un noeud
            # On continue l'exploration en respectant la règle pour obtenir le code :
            # Gauche -> 0, droite -> 1
            fileExploration.append((code + "0", truc[1]))
            fileExploration.append((code + "1", truc[2]))
            l += 2

    return dico

def huffman(liste):
    """
    On se contente de remplacer les lettres du texte par le code binaire
    obtenu à l'aide de la fonction creerDico.
    """
    alphabet = compterOccurences(liste)
    arbre = creerArbre(alphabet)
    dico = creerDico(arbre)
    texteCompresse = ""
    for i in liste:
        if type(i)==list:
            texteCompresse += dico["$"+str(i[1])]
        else:
            texteCompresse += dico[i]
    # On n'oublie pas de renvoyer aussi le dictionnaire,
    # sinon il sera impossible de décompresser le texte
    return texteCompresse, dico

def huffman_inverse(texteCompresse, dicoRetourne):
    """
    Décompresse un texte à l'aide de son dico.
    Une fois encore, on utilise une file. C'est un outil très puissant
    qui permet de ne jamais écrire de fonction récursive. Chaque élément
    de la file est un 2-uplet, le premier élément est le texte décompressé
    jusque là, le second est le code binaire restant à décompresser.
    """
    # On retourne le dico
    dico = {v: k for (k, v) in dicoRetourne.items()}
    # Nombre maximum de bits d'un caractère compressé
    limite = max(len(k) for k in dico.keys())
    fileExploration = [("", texteCompresse)]
    l = 1
    a=[]
    while l >= 1:
        fait, restant = fileExploration.pop(0)  # On défile le premier élément
        l -= 1
        # On regarde si la décompression est terminée
        if restant == "":
            return a
        # Sinon, on tente de remplacer les i premiers bits de restant par un caractère
        i = 0
        bits = ""
        for bit in restant:
            bits += bit
            i += 1
            if i > limite:
                # C'est pas la peine de continuer, bits est trop long
                # pour correspondre à un caractère
                break
            elif bits in dico:
                # On a la possibilité de remplacer quelques 0 et 1 par un caractère
                # alors on le fait, sans pour autant considérer que l'on a choisi
                # le bon remplacement
                if type(dico[bits])==str:
                    a.append([0,int(dico[bits][1:])])
                else:
                    a.append(dico[bits])
                fileExploration.append((fait+str(dico[bits]), restant[i:]))
                l += 1
                
                # Puis on continue à explorer les possibilités
    # Aucune décompression n'a fonctionné, on ne renvoie rien
    return None



In [247]:
#Quantization Matrix from https://hal.science/hal-03858141/file/qtable.pdf
#Quantification minimale
Q1=[1,1,1,1,1,2,2,2,1,1,1,1,1,2,2,2,1,1,1,1,2,2,3,2,1,1,1,1,2,3,3,2,1,1,1,2,3,4,4,3,1,1,2,3,3,4,5,4,2,3,3,3,4,5,5,4,3,4,4,4,4,4,4,4]
#Quantification intermédiaire
Q2=[1,2,2,3,5,8,10,12,2,2,3,4,5,12,12,11,3,3,3,5,8,11,14,11,3,3,4,6,10,17,16,12,4,4,7,11,14,22,21,15,5,7,11,13,16,21,23,18,10,13,16,17,21,24,24,20,14,18,19,20,22,20,21,20]
#Quantification importante
Q3=[1,9,8,13,19,32,41,49,10,10,11,15,21,46,48,255,11,10,13,19,32,46,55,45,11,14,18,23,41,255,255,255,14,18,30,45,54,255,255,255,19,28,44,51,255,255,255,255,39,51,62,255,255,255,255,255,58,255,255,255,255,255,255,255]

Q1=np.array(Q1)
Q1=Q1.reshape((8,8))
Q2=np.array(Q2)
Q2=Q2.reshape((8,8))
Q3=np.array(Q3)
Q3=Q3.reshape((8,8))
def quantification(l,Q):
    return np.round(l*Q)


In [248]:
#Extraction des données

Table=[]
def Une_fonction_pour_les_rassembler_toute(n,T):
    title=["Black&White","Arbre","Montagne","Mère","Gravure","Libellulle","Kayak","Lunch","Eiffel","Maison","Poule","Poème","Fleur"]
    path="image_test/"+str(n)+".jpg" 
    image_test=Image_couleur(path).divisionen3().luminance
    Test=Image(title[n],image_test)
    Test.division()
    Test.compressI(Q2)
    Test.decompressI(Q2,T)
    return Test

Une_fonction_pour_les_rassembler_toute(6,True).metric(1,1,False)



"""
coefficient=[]
for j in range(1,13):
    Test=Une_fonction_pour_les_rassembler_toute(j,False)
    coefficient.append(Test.coeffcount())
print(coefficient)



Table=[]
for k in range(1,4):
    Table.append([])
    if k==1:
        Q=Q1
    elif k==2:
        Q=Q2
    else:
        Q=Q3
    for i in range(1,13):
        Table[k-1].append([])
        for j in range(1,13):
            Test=Une_fonction_pour_les_rassembler_toute(j,False)
            Test.metric(k-1,i-1,True)
        print((i/12)*100)
print (Table)

"""

Avant : 2000000 bits / Après : 342596 bits,
 Taux de compression: 0.17
Erreur Moyenne: 0.258 
 Variance de l'erreur: 0.102


'\ncoefficient=[]\nfor j in range(1,13):\n    Test=Une_fonction_pour_les_rassembler_toute(j,False)\n    coefficient.append(Test.coeffcount())\nprint(coefficient)\n\n\n\nTable=[]\nfor k in range(1,4):\n    Table.append([])\n    if k==1:\n        Q=Q1\n    elif k==2:\n        Q=Q2\n    else:\n        Q=Q3\n    for i in range(1,13):\n        Table[k-1].append([])\n        for j in range(1,13):\n            Test=Une_fonction_pour_les_rassembler_toute(j,False)\n            Test.metric(k-1,i-1,True)\n        print((i/12)*100)\nprint (Table)\n\n'

In [249]:
#Génération d'images test, en damier noir/blanc, avec un gradient fort avec un gradient plus faible, au hasard
"""
C=[]
for i in range(1,100):
    C.append([])
    for j in range(1,100):
        C[i-1].append(((i+j)%2)*255)
G=[]
for i in range(1,100):
    G.append([])
    for j in range(1,100):
        G[i-1].append((np.cos((i+j)*0.5)+1)*255)
S=[]
for i in range(1,100):
    S.append([])
    for j in range(1,100):
        S[i-1].append((np.cos((i+j)*0.05)+1)*255)     
R=[]
for i in range(1,100):
    R.append([])
    for j in range(1,100):
        R[i-1].append(random.randint(0,255))

coefficient=[]
for j in [C,G,S,R]:
    
    Test=Image("Standard",j)
    Test.division()
    Test.compressI(Q3)
    Test.decompressI(Q3, True)
    coefficient.append(Test.coeffcount())
print(coefficient)
"""


'\nC=[]\nfor i in range(1,100):\n    C.append([])\n    for j in range(1,100):\n        C[i-1].append(((i+j)%2)*255)\nG=[]\nfor i in range(1,100):\n    G.append([])\n    for j in range(1,100):\n        G[i-1].append((np.cos((i+j)*0.5)+1)*255)\nS=[]\nfor i in range(1,100):\n    S.append([])\n    for j in range(1,100):\n        S[i-1].append((np.cos((i+j)*0.05)+1)*255)     \nR=[]\nfor i in range(1,100):\n    R.append([])\n    for j in range(1,100):\n        R[i-1].append(random.randint(0,255))\n\ncoefficient=[]\nfor j in [C,G,S,R]:\n\n    Test=Image("Standard",j)\n    Test.division()\n    Test.compressI(Q3)\n    Test.decompressI(Q3, True)\n    coefficient.append(Test.coeffcount())\nprint(coefficient)\n'